In [6]:
import os

os.environ["DGLBACKEND"] = "pytorch"
from functools import partial

import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from dgl.nn import GATConv
import numpy as np
from pygod.utils import load_data as pygod_load_data

In [2]:
class RGCNLayer(nn.Module):
    def __init__(
        self,
        in_feat,
        out_feat,
        num_rels,
        num_bases=-1,
        bias=None,
        activation=None,
        is_input_layer=False,
    ):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        # sanity check
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels
        # weight bases in equation (3)
        self.weight = nn.Parameter(
            torch.Tensor(self.num_bases, self.in_feat, self.out_feat)
        )
        if self.num_bases < self.num_rels:
            # linear combination coefficients in equation (3)
            self.w_comp = nn.Parameter(
                torch.Tensor(self.num_rels, self.num_bases)
            )
        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))
        # init trainable parameters
        nn.init.xavier_uniform_(
            self.weight, gain=nn.init.calculate_gain("relu")
        )
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(
                self.w_comp, gain=nn.init.calculate_gain("relu")
            )
        if self.bias:
            nn.init.xavier_uniform_(
                self.bias, gain=nn.init.calculate_gain("relu")
            )

    def forward(self, g):
        if self.num_bases < self.num_rels:
            # generate all weights from bases (equation (3))
            weight = self.weight.view(
                self.in_feat, self.num_bases, self.out_feat
            )
            weight = torch.matmul(self.w_comp, weight).view(
                self.num_rels, self.in_feat, self.out_feat
            )
        else:
            weight = self.weight
        if self.is_input_layer:

            def message_func(edges):
                # for input layer, matrix multiply can be converted to be
                # an embedding lookup using source node id
                embed = weight.view(-1, self.out_feat)
                index = edges.data[dgl.ETYPE] * self.in_feat + edges.src["id"]
                return {"msg": embed[index] * edges.data["norm"]}

        else:

            def message_func(edges):
                w = weight[edges.data[dgl.ETYPE]]
                msg = torch.bmm(edges.src["h"].unsqueeze(1), w).squeeze()
                msg = msg * edges.data["norm"]
                return {"msg": msg}

        def apply_func(nodes):
            h = nodes.data["h"]
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {"h": h}

        g.update_all(message_func, fn.sum(msg="msg", out="h"), apply_func)

In [3]:
class Model(nn.Module):
    def __init__(
        self,
        num_nodes,
        h_dim,
        out_dim,
        num_rels,
        num_bases=-1,
        num_hidden_layers=1,
    ):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for _ in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer()
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        features = torch.arange(self.num_nodes)
        return features

    def build_input_layer(self):
        return RGCNLayer(
            self.num_nodes,
            self.h_dim,
            self.num_rels,
            self.num_bases,
            activation=F.relu,
            is_input_layer=True,
        )

    def build_hidden_layer(self):
        return RGCNLayer(
            self.h_dim,
            self.h_dim,
            self.num_rels,
            self.num_bases,
            activation=F.relu,
        )

    def build_output_layer(self):
        return RGCNLayer(
            self.h_dim,
            self.out_dim,
            self.num_rels,
            self.num_bases,
            activation=partial(F.softmax, dim=1),
        )

    def forward(self, g):
        if self.features is not None:
            g.ndata["id"] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop("h")

# Amazon

In [4]:
from dgl.data import FraudAmazonDataset
amazon = FraudAmazonDataset()
g1 = amazon[0]
num_classes = amazon.num_classes
feat = g1.ndata['feature']
label = g1.ndata['label']

Done loading data from cached files.


In [5]:
train_mask = g1.nodes['user'].data['train_mask']
test_mask = g1.nodes['user'].data['test_mask']
train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze()
test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze()
labels = g1.nodes['user'].data.pop("label")
num_rels = len(g1.canonical_etypes)
num_classes = amazon.num_classes

# normalization factor
for cetype in g1.canonical_etypes:
    g1.edges[cetype].data["norm"] = dgl.norm_by_dst(g1, cetype).unsqueeze(1)

In [6]:
# configurations
n_hidden = 16  # number of hidden units
n_bases = -1  # use number of relations as number of bases
n_hidden_layers = 0  # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 100  # epochs to train
lr = 0.01  # learning rate
l2norm = 0  # L2 norm coefficient

# create graph
g = dgl.to_homogeneous(g1, edata=["norm"])
node_ids = torch.arange(g.num_nodes())
#target_idx = node_ids[g1.ndata[dgl.NTYPE] == category_id]

# create model
model = Model(
    g.num_nodes(),
    n_hidden,
    num_classes,
    num_rels,
    num_bases=n_bases,
    num_hidden_layers=n_hidden_layers,
)

In [7]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    #logits = logits[target_idx]
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[test_idx], labels[test_idx])
    val_acc = torch.sum(logits[test_idx].argmax(dim=1) == labels[test_idx])
    val_acc = val_acc.item() / len(test_idx)
    print(
        "Epoch {:05d} | ".format(epoch)
        + "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
            train_acc, loss.item()
        )
        + "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
            val_acc, val_loss.item()
        )
    )

start training...
Epoch 00000 | Train Accuracy: 0.5472 | Train Loss: 0.6932 | Validation Accuracy: 0.5402 | Validation loss: 0.6932
Epoch 00001 | Train Accuracy: 0.9054 | Train Loss: 0.6649 | Validation Accuracy: 0.8936 | Validation loss: 0.6664
Epoch 00002 | Train Accuracy: 0.9054 | Train Loss: 0.6351 | Validation Accuracy: 0.8936 | Validation loss: 0.6384
Epoch 00003 | Train Accuracy: 0.9054 | Train Loss: 0.6028 | Validation Accuracy: 0.8936 | Validation loss: 0.6080
Epoch 00004 | Train Accuracy: 0.9054 | Train Loss: 0.5697 | Validation Accuracy: 0.8936 | Validation loss: 0.5768
Epoch 00005 | Train Accuracy: 0.9054 | Train Loss: 0.5374 | Validation Accuracy: 0.8936 | Validation loss: 0.5462
Epoch 00006 | Train Accuracy: 0.9054 | Train Loss: 0.5077 | Validation Accuracy: 0.8936 | Validation loss: 0.5180
Epoch 00007 | Train Accuracy: 0.9054 | Train Loss: 0.4820 | Validation Accuracy: 0.8936 | Validation loss: 0.4934
Epoch 00008 | Train Accuracy: 0.9054 | Train Loss: 0.4610 | Validation

In [8]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

# After training, when evaluating on test data:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    logits = model.forward(g)
    probabilities = F.softmax(logits[test_idx], dim=1)  # Convert logits to probabilities
    # Assuming your positive class is 1 (adjust accordingly if it's different)
    positive_probabilities = probabilities[:, 1]  # Get probabilities for the positive class
    # Calculate ROC-AUC
    roc_auc = roc_auc_score(labels[test_idx].cpu(), positive_probabilities.cpu())
    print(f"ROC-AUC Score: {roc_auc}")

ROC-AUC Score: 0.8507176023638665


# Yelp

In [9]:
from dgl.data import FraudYelpDataset
yelp = FraudYelpDataset()
g2 = yelp[0]
num_classes = yelp.num_classes
feat = g2.ndata['feature']
label = g2.ndata['label']
train_mask = g2.ndata['train_mask']
test_mask = g2.ndata['test_mask']

Done loading data from cached files.


In [10]:
train_mask = g2.nodes['review'].data['train_mask']
test_mask = g2.nodes['review'].data['test_mask']
train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze()
test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze()
labels = g2.nodes['review'].data.pop("label")
num_rels = len(g2.canonical_etypes)
num_classes = amazon.num_classes

# normalization factor
for cetype in g2.canonical_etypes:
    g2.edges[cetype].data["norm"] = dgl.norm_by_dst(g2, cetype).unsqueeze(1)

In [11]:
# configurations
n_hidden = 16  # number of hidden units
n_bases = -1  # use number of relations as number of bases
n_hidden_layers = 0  # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 100  # epochs to train
lr = 0.01  # learning rate
l2norm = 0  # L2 norm coefficient

# create graph
g = dgl.to_homogeneous(g2, edata=["norm"])
node_ids = torch.arange(g.num_nodes())
#target_idx = node_ids[g2.ndata[dgl.NTYPE] == category_id]

# create model
model = Model(
    g.num_nodes(),
    n_hidden,
    num_classes,
    num_rels,
    num_bases=n_bases,
    num_hidden_layers=n_hidden_layers,
)

In [12]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    #logits = logits[target_idx]
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[test_idx], labels[test_idx])
    val_acc = torch.sum(logits[test_idx].argmax(dim=1) == labels[test_idx])
    val_acc = val_acc.item() / len(test_idx)
    print(
        "Epoch {:05d} | ".format(epoch)
        + "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
            train_acc, loss.item()
        )
        + "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
            val_acc, val_loss.item()
        )
    )

start training...
Epoch 00000 | Train Accuracy: 0.6470 | Train Loss: 0.6930 | Validation Accuracy: 0.6386 | Validation loss: 0.6930
Epoch 00001 | Train Accuracy: 0.8631 | Train Loss: 0.6745 | Validation Accuracy: 0.8563 | Validation loss: 0.6769
Epoch 00002 | Train Accuracy: 0.8639 | Train Loss: 0.6525 | Validation Accuracy: 0.8589 | Validation loss: 0.6581
Epoch 00003 | Train Accuracy: 0.8640 | Train Loss: 0.6280 | Validation Accuracy: 0.8611 | Validation loss: 0.6367
Epoch 00004 | Train Accuracy: 0.8641 | Train Loss: 0.6023 | Validation Accuracy: 0.8626 | Validation loss: 0.6136
Epoch 00005 | Train Accuracy: 0.8637 | Train Loss: 0.5765 | Validation Accuracy: 0.8628 | Validation loss: 0.5896
Epoch 00006 | Train Accuracy: 0.8634 | Train Loss: 0.5519 | Validation Accuracy: 0.8634 | Validation loss: 0.5660
Epoch 00007 | Train Accuracy: 0.8625 | Train Loss: 0.5296 | Validation Accuracy: 0.8628 | Validation loss: 0.5437
Epoch 00008 | Train Accuracy: 0.8623 | Train Loss: 0.5102 | Validation

In [13]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

# After training, when evaluating on test data:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    logits = model.forward(g)
    probabilities = F.softmax(logits[test_idx], dim=1)  # Convert logits to probabilities
    # Assuming your positive class is 1 (adjust accordingly if it's different)
    positive_probabilities = probabilities[:, 1]  # Get probabilities for the positive class
    # Calculate ROC-AUC
    roc_auc = roc_auc_score(labels[test_idx].cpu(), positive_probabilities.cpu())
    print(f"ROC-AUC Score: {roc_auc}")

ROC-AUC Score: 0.7221614390424578


# Reddit

In [2]:
# Graph Convolutional Network¶

In [7]:
reddit = pygod_load_data('reddit')
g3 = dgl.graph((reddit.edge_index[0], reddit.edge_index[1]))
g3.ndata['feature'] = reddit.x
g3.ndata['label'] = reddit.y.type(torch.LongTensor)

In [8]:
num_nodes = g3.number_of_nodes()
indices = np.random.permutation(num_nodes)

# Assuming 70% training, 15% validation, 15% testing split
train_size = int(num_nodes * 0.8)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[indices[:train_size]] = True
test_mask[indices[train_size:]] = True

# Assign the masks to your graph
g3.ndata['train_mask'] = train_mask
g3.ndata['test_mask'] = test_mask

In [12]:
class GATModel(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads):
        super(GATModel, self).__init__()
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads)
        # Assuming we concatenate the heads in the first layer, the input dimension to the second layer is hidden_dim * num_heads
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=1)  # Output layer

    def forward(self, g, features):
        h = self.layer1(g, features)
        h = torch.flatten(h, start_dim=1)  # Flatten the multi-head outputs for concatenation
        h = F.elu(h)
        h = self.layer2(g, h)
        h = h.mean(dim=1)  # Aggregate multi-head outputs if needed
        return h

In [15]:
def train(model, g, features, labels, train_mask, epochs, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')

        
model = GATModel(in_dim=g3.ndata['feature'].shape[1], hidden_dim=8, out_dim=2, num_heads=4)
train(model, g3, g3.ndata['feature'], g3.ndata['label'], train_mask, epochs=100, lr=0.005)

Epoch 1/100, Loss: 0.7153367400169373
Epoch 2/100, Loss: 0.6369478702545166
Epoch 3/100, Loss: 0.5658162832260132
Epoch 4/100, Loss: 0.5017507076263428
Epoch 5/100, Loss: 0.4444631338119507
Epoch 6/100, Loss: 0.39368775486946106
Epoch 7/100, Loss: 0.34911876916885376
Epoch 8/100, Loss: 0.3104305863380432
Epoch 9/100, Loss: 0.2772606611251831
Epoch 10/100, Loss: 0.2491980642080307
Epoch 11/100, Loss: 0.22578692436218262
Epoch 12/100, Loss: 0.20654098689556122
Epoch 13/100, Loss: 0.19096189737319946
Epoch 14/100, Loss: 0.1785585582256317
Epoch 15/100, Loss: 0.16886256635189056
Epoch 16/100, Loss: 0.16144034266471863
Epoch 17/100, Loss: 0.1558997631072998
Epoch 18/100, Loss: 0.15189380943775177
Epoch 19/100, Loss: 0.14912071824073792
Epoch 20/100, Loss: 0.1473221778869629
Epoch 21/100, Loss: 0.14627999067306519
Epoch 22/100, Loss: 0.1458122283220291
Epoch 23/100, Loss: 0.14576861262321472
Epoch 24/100, Loss: 0.14602665603160858
Epoch 25/100, Loss: 0.1464875042438507
Epoch 26/100, Loss: 0.

In [16]:
from sklearn.metrics import roc_auc_score

def evaluate(model, g, features, labels, test_mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        probs = torch.softmax(logits[test_mask], dim=1)[:, 1].numpy()  # Probability of the positive class
        auc_score = roc_auc_score(labels[test_mask].numpy(), probs)
        print(f'Test ROC-AUC: {auc_score}')

# Example usage
evaluate(model, g3, g3.ndata['feature'], g3.ndata['label'], test_mask)

Test ROC-AUC: 0.6209628139679952
